In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()["Outcome"].sort_values()

,Outcome
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
DiabetesPedigreeFunction,0.173844
Pregnancies,0.221898
Age,0.238356
BMI,0.292695
Glucose,0.466581
Outcome,1.000000


In [6]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [13]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input

In [17]:
model = Sequential([
    Input(shape=(8,)),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [23]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.8057 - loss: 0.4056 - val_accuracy: 0.8117 - val_loss: 0.4522
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8166 - loss: 0.3976 - val_accuracy: 0.8182 - val_loss: 0.4509
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7946 - loss: 0.4271 - val_accuracy: 0.8182 - val_loss: 0.4536
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8065 - loss: 0.3925 - val_accuracy: 0.8182 - val_loss: 0.4541
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8126 - loss: 0.4031 - val_accuracy: 0.8182 - val_loss: 0.4550
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8142 - loss: 0.4121 - val_accuracy: 0.8182 - val_loss: 0.4548
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8042 - loss: 0.4047 - val_accuracy: 0.8182 - val_loss: 0.4535
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8316 - loss: 0.3699 - val_accuracy: 0.8182 

In [27]:
import kerastuner as kt

In [36]:
from keras.src.optimizers import optimizer
def build_model(hp):

  model = Sequential()
  model.add(Dense(32, activation="relu", input_shape=(8,)))
  model.add(Dense(1, activation="sigmoid"))

  optimizer = hp.Choice('optimzer', values=['adam', 'sgd', 'rmsprop','adadelta'])
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [37]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [38]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 01m 54s


In [40]:
tuner.get_best_hyperparameters()[0].values

{'optimzer': 'rmsprop'}

In [42]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [43]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.fit(X_train, y_train, batch_size=32,initial_epoch=6, epochs=100, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.7430 - loss: 0.5391 - val_accuracy: 0.8052 - val_loss: 0.4901
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7483 - loss: 0.5110 - val_accuracy: 0.8052 - val_loss: 0.4798
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7490 - loss: 0.5064 - val_accuracy: 0.7987 - val_loss: 0.4716
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7461 - loss: 0.5009 - val_accuracy: 0.7987 - val_loss: 0.4661
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7709 - loss: 0.4716 - val_accuracy: 0.7922 - val_loss: 0.4613
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7614 - loss: 0.4802 - val_accuracy: 0.7987 - val_loss: 0.4581
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7690 - loss: 0.4759 - val_accuracy: 0.8052 - val_loss: 0.4567
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7644 - loss: 0.4779 - val_accuracy: 0.81

In [52]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units', min_value=8, max_value=128, step=8)
  model.add(Dense(units, activation="relu", input_shape=(8,)))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [53]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='some_directory')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 21s


In [55]:
tuner.get_best_hyperparameters()[0].values

{'units': 56}

In [56]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [58]:
model.fit(X_train, y_train, batch_size=32,initial_epoch=6, epochs=100, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.7505 - loss: 0.5040 - val_accuracy: 0.7922 - val_loss: 0.4855
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7647 - loss: 0.4906 - val_accuracy: 0.7857 - val_loss: 0.4719
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7776 - loss: 0.4627 - val_accuracy: 0.7857 - val_loss: 0.4642
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7772 - loss: 0.4625 - val_accuracy: 0.7922 - val_loss: 0.4574
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7817 - loss: 0.4669 - val_accuracy: 0.7922 - val_loss: 0.4523
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7550 - loss: 0.4693 - val_accuracy: 0.7922 - val_loss: 0.4503
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7548 - loss: 0.4773 - val_accuracy: 0.7987 - val_loss: 0.4476
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7534 - loss: 0.4595 - val_accuracy: 0.79

In [66]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72,activation='relu',input_shape=(8,)))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [68]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir')

In [71]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 28s


In [72]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [74]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [75]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - accuracy: 0.8014 - loss: 0.4270 - val_accuracy: 0.7662 - val_loss: 0.4807
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8100 - loss: 0.4214 - val_accuracy: 0.7403 - val_loss: 0.6288
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8314 - loss: 0.4007 - val_accuracy: 0.7792 - val_loss: 0.4767
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8617 - loss: 0.3559 - val_accuracy: 0.7792 - val_loss: 0.4794
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8089 - loss: 0.3903 - val_accuracy: 0.7922 - val_loss: 0.4918
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7995 - loss: 0.3942 - val_accuracy: 0.7597 - val_loss: 0.5143
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8026 - loss: 0.3688 - val_accuracy: 0.7597 - val_loss: 0.5874
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8412 - loss: 0.3709 - val_accuracy: 0.785

In [ ]:
def build_model(hp):
  model = Sequential()

  counter = 0
  units = hp.Int("")

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128, step=8), activation='rmsprop'))